<a href="https://colab.research.google.com/github/afiaka87/deep-daze/blob/notebook/sleepy_daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sleepy-daze

## 0. Install Dependencies - (Runtime > Restart Runtime after running this cell)

In [ ]:
#@title Restart runtime after this finishes.
import subprocess
import re

pytorch_version = "1.7.1" + "+cu101"
torchvision_version = "0.8.2" +  "+cu101"

!pip install torch=={pytorch_version} torchvision=={torchvision_version} -f https://download.pytorch.org/whl/torch_stable.html
print("RESTART THE RUNTIME!")

## 1. Define global variables (rerun as needed)

In [ ]:
text = 'The sirens of titan' # @param {type:"string"}
image_width = "512" #@param ["128", "256", "512", ""]
image_width = int(image_width)
save_every = 1 # @param {type:"integer"}
save_progress = True # @param {type:"boolean"}
seed = 657345 # @param {type:"integer"}

## 2. Generate an image from text with big-sleep

In [ ]:


%mkdir /content/bs_temp
%cd /content/bs_temp
!pip install regex deep-daze big-sleep

bs_learning_rate = 6e-2 # @param {type:"number"}
bs_iterations = 500 # @param {type:"integer"}

import big_sleep
import torch

imagine = None

imagine = big_sleep.Imagine(
    image_size=image_width,
    text=text,
    save_every=save_every,
    lr=bs_learning_rate,
    save_best=True,
    iterations=bs_iterations,
    epochs=1,
    save_progress=save_progress,
    seed=seed
)
imagine()
del imagine
imagine = None
torch.cuda.empty_cache()

%cd /content

# 3. Prime deep-daze with big-sleep's output. 
- big-sleep seems to leak some memory, so you may need to restart the runtime, and rerun [1]

- Leave change_text empty to use global variable.
- 26 layers is about as high as you can go in colab.


In [ ]:
import torch

torch.cuda.empty_cache()

%mkdir /content/dd_temp
%cd /content/dd_temp

import deep_daze
start_image_path = text.replace(" ", "_") + ".best.png"
!cp /content/bs_temp/{start_image_path} /content/dd_temp


change_text = "" #@param {type: "string"}
if change_text is not "":
   text = change_text
dd_num_layers = 26 # @param {type:"integer"}
dd_learning_rate = 1e-5  # @param {type: "number"}
dd_iterations = 500 # @param {type:"integer"}
dd_epochs = 1  # @param {type:"integer"}
initial_learning_rate = 2e-4  # @param {type:"number"}
initial_iterations =  100# @param {type: "integer"}
dd_batch_size = 8 # @param {type: "integer"}
dd_gradient_accumulate_every = 1 # @param {type: "integer"}

imagine = None # Clears the GPU memory hopefully...

imagine = deep_daze.Imagine(
    text,
    lr=dd_learning_rate,
    num_layers=dd_num_layers,
    batch_size=dd_batch_size,
    gradient_accumulate_every=dd_gradient_accumulate_every,
    epochs=dd_epochs,
    iterations=dd_iterations,
    image_width=image_width,
    save_every=save_every,
    save_progress=save_progress,
    seed=seed,
    open_folder=False,
    save_date_time=False,
    start_image_path=start_image_path,
    start_image_train_iters=initial_iterations,
    theta_initial=30,
    theta_hidden=30,
    start_image_lr=initial_learning_rate,
)
imagine()

del imagine

torch.cuda.empty_cache()

%cd /content

# 4. Create a video of the whole thing
- filename **must end in .mp4**
- higher crf for more compression

In [ ]:
fps = 30 #@param {type:"integer"}
crf = 5 #@param {type:"integer"}
filename = "sleepy-daze.mp4" #@param {type:"string"}
text_with_underscores = text.replace(" ", "_")
%cd /content/bs_temp
bs_ffmpeg_fmt_str = text_with_underscores + ".%d.png"
bs_output_video = text.replace(" ", "_") + ".mp4"
!ffmpeg -r {fps} -i {bs_ffmpeg_fmt_str} -pix_fmt yuv420p -crf {crf} -y {bs_output_video}

%cd /content/dd_temp
dd_ffmpeg_fmt_str = text_with_underscores + ".%06d.png"
dd_output_video = text.replace(" ", "_") + ".mp4"
!ffmpeg -r 30 -i {dd_ffmpeg_fmt_str} -pix_fmt yuv420p -crf {crf} -y {bs_output_video}
%cd /content
!echo file /content/bs_temp/{bs_output_video} > file_order.txt
!echo file /content/dd_temp/{dd_output_video} >> file_order.txt
!ffmpeg -f concat -safe 0 -i file_order.txt -c copy {filename}
